In [32]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures
import time
import random

## Group Assignment
### Team Number: 15
### Team Member Names: Daniel Eric Jodie
### Team Strategy Chosen: SAFE

In [33]:
##bing s
start = time.perf_counter()
def get_info(stock):
    temp_frame=pd.DataFrame()
    tick = yf.Ticker(stock)
    tick_hist = tick.history(start=start_date,end=end_date)
    if len(tick_hist.index) != 0:
        tick_info = tick.info
        if tick_info['currency'] == 'USD':
            if tick_hist['Volume'].mean()>= 10000:
                for need in hist_need:
                    temp_frame[need+' of '+stock]=tick_hist[need]
                return (temp_frame,stock, tick_info['sector'])
    return pd.DataFrame(),'',''
start_date = '2021-07-02'
end_date = '2021-10-22'
hist_data = pd.DataFrame()
clean_stocks = []
industry_list = []
hist_need = ['Open','High','Low','Close','Volume']
tick_data = pd.read_csv("Tickers.csv",names=["Tickers"])
#with concurrent.futures.ThreadPoolExecutor() as executor:
list_tick = []
for x in range(len(tick_data.index)):
    list_tick.append(tick_data['Tickers'].iloc[x])
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_info, list_tick)
    for result in results:
        hist_data = pd.concat([hist_data,result[0]],join='outer',axis=1)
        clean_stocks.append(result[1])
        industry_list.append(result[2])
while True:
    if "" not in clean_stocks:
        break
    else:
        industry_list.remove("")
        clean_stocks.remove("")
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds(s)')
hist_data.dropna(inplace=True)
hist_data.head()

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
Finished in 16.7 seconds(s)


,Open of AAPL,High of AAPL,Low of AAPL,Close of AAPL,Volume of AAPL,Open of ABBV,High of ABBV,Low of ABBV,Close of ABBV,Volume of ABBV,...,Open of UPS,High of UPS,Low of UPS,Close of UPS,Volume of UPS,Open of USB,High of USB,Low of USB,Close of USB,Volume of USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-02,137.493033,139.586842,137.343482,139.546967,78852600.0,111.900746,112.848562,111.285158,112.535881,5719500.0,...,207.463349,209.947891,207.156497,209.383667,2172300.0,57.171395,57.300406,56.744668,57.131699,3520900.0
2021-07-06,139.656634,142.727531,139.656634,141.600876,108181800.0,112.535879,113.249187,112.145027,113.083076,7172100.0,...,208.324538,209.462863,206.077567,209.146103,2421300.0,56.665274,56.893526,55.206468,55.583572,5008700.0
2021-07-07,143.116382,144.462404,142.238989,144.143356,104911600.0,113.258966,114.294720,112.672683,114.079750,6695100.0,...,209.165904,212.749168,208.185959,212.085968,2553700.0,55.246161,56.049996,55.037761,55.563728,5125700.0
2021-07-08,141.162171,143.634848,140.254853,142.817276,105575500.0,113.112394,114.138374,113.034222,113.688896,3948800.0,...,208.185954,210.541795,207.770203,209.918198,2354300.0,54.700349,55.216391,54.253774,54.620960,5977700.0
2021-07-09,142.328727,145.220162,142.229016,144.681763,99890800.0,113.640040,114.646482,113.454384,113.913635,4823400.0,...,211.719726,212.323537,210.512104,211.749420,2050500.0,55.653043,56.605733,55.424791,56.536266,4437300.0


In [34]:
def get_daily_returns(list_of_stock):
    monthly_returns = pd.DataFrame()
    for stock in list_of_stock:
        monthly_returns['Monthly Returns of '+stock] = hist_data['Close of '+stock].pct_change()*100
    monthly_returns.dropna(inplace=True)
    return monthly_returns

def get_beta_total(list_of_stock):
    tot = 0
    for x in range (len(list_of_stock)):
        tot+= beta_vals['Monthly Returns of '+list_of_stock[x]]
    return (tot/len(list_of_stock)).iloc[0]

def get_average_std(list_of_stock):
    tot = 0
    for x in range (len(list_of_stock)):
        tot+= monthly_returns['Monthly Returns of '+list_of_stock[x]].std(axis=0,skipna=True)
    return (tot/len(list_of_stock))

def get_alpha(list_of_stock):
    # Get the current beta column
    column_counter = 1
    alpha = pd.DataFrame()
    for stock in list_of_stock:
        alpha['Alpha '+stock] = monthly_returns['Monthly Returns of '+stock] - ((monthly_returns['Monthly Returns of '+stock].mean()) * beta_vals.iloc[0, column_counter])
        column_counter += 1
    return alpha

# MDD Equation: (Peak Value - Trough Value) / Peak Value
def get_mdd(list_of_stock):
    mdd = []
    for x in range (len(list_of_stock)):
        trough = min(hist_data['Low of '+list_of_stock[x]])
        peak = max(hist_data['High of '+list_of_stock[x]])
        mdd.append((((peak - trough) / peak)))
    return mdd

In [35]:
monthly_returns = get_daily_returns(clean_stocks)
monthly_returns.head()

,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,Monthly Returns of BIIB,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-06,1.471841,0.486241,0.660788,0.183690,-2.996878,4.692707,0.629081,-0.228153,-2.624553,-1.098299,...,-4.940534,0.178778,-1.535144,-0.718435,-0.458268,-0.525472,0.219863,-0.404951,-0.113459,-2.709750
2021-07-07,1.795526,0.881364,0.883694,1.378523,0.386187,0.566963,0.837459,-1.846363,-0.798599,3.027044,...,-1.572021,0.081111,-1.449386,-0.310134,0.855572,-0.742682,0.341241,2.028508,1.405651,-0.035701
2021-07-08,-0.919973,-0.342615,-0.508882,-0.148561,-4.124817,0.942218,-2.035319,2.152906,-2.440249,3.861200,...,-0.488914,-0.226939,-0.556491,-0.809573,0.416077,-0.811455,-0.415389,-4.383626,-1.022119,-1.696733
2021-07-09,1.305505,0.197679,0.402476,1.122420,4.480600,-0.323466,2.650744,1.191026,3.249103,-2.950815,...,2.161804,-0.129972,3.677358,0.958130,0.000000,1.078407,0.526891,1.534306,0.872351,3.506540
2021-07-12,-0.420381,0.900663,-0.776685,-0.335861,1.216133,-0.021241,0.965453,-0.542595,1.473522,-2.546347,...,0.096184,0.536845,0.447218,0.105440,1.472372,0.867193,0.953630,-0.212007,-0.275798,1.070743


In [36]:
beta_data = pd.DataFrame()
MarketIndex='^GSPC'
marketing_tick = yf.Ticker(MarketIndex)
marketing_hist = marketing_tick.history(start=start_date,end=end_date)
beta_data[MarketIndex] = marketing_hist['Close'].pct_change()*100
beta_data = pd.concat([beta_data, monthly_returns],join='inner',axis=1)
MarketVar= beta_data['^GSPC'].var()
Beta = pd.DataFrame(beta_data.cov() / MarketVar)
beta_vals = pd.DataFrame(Beta.iloc[[0]])
beta_vals.rename(index={'^GSPC': 'Beta'})
beta_vals.head()

,^GSPC,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
^GSPC,1.0,1.29928,0.668293,0.458501,1.044191,1.484594,1.010834,1.425493,1.443641,1.422094,...,1.673033,0.295346,1.557336,0.67174,0.985831,1.031316,0.91933,1.065526,0.703829,1.257542


We used beta to measure the systematic risk (ie. volatility) of our portfolio value. The beta coefficient measures the volatility of an individual stock relative to the entire market’s systematic risk. This is effective in displaying the activity of a stock’s returns in reaction to rises and dips in the market. As such, it allows us to determine how risky a stock is in comparison to the rest of the market. With our portfolio’s safe strategy, we look for stocks with beta coefficients as close to 0 as possible. A beta coefficient close to 0 deduces that such stocks deviate very little from the market trends and thus, does not add much risk to the portfolio. The stocks that we look to invest in have low monthly returns which means that there is less at stakes. With low-risk investing, targeting stocks with low monthly returns means that we are protected against any possible loss.  

An assumption made of the beta coefficient theory is that the stock returns are normally distributed. However, financial markets are susceptible to unavoidable shock events. As a matter of fact, returns are rarely normally distributed. Thus, the beta coefficient of a stock may be inaccurate when predicting its future movement. For instance, a stock has small price swings (low std) but is in a long-term downward trend. Thus, though the stock is determined as low risk by the beta coefficient, the potential for losses is, in contrary, relatively high. 

Furthermore, a limitation of using beta is that it is mostly accurate and effective in determining short-term risk. Since we calculate beta solely using historical closing prices, it is not as practical in aims of predicting the stock’s future movement. However, 
since the time interval is only 3 months, we concluded that beta is, nevertheless, useful in calculating a stock’s volatility.


In [37]:
alpha = pd.DataFrame()
for stock in clean_stocks:
    alpha['Alpha '+stock] = monthly_returns['Monthly Returns of '+stock] - ((monthly_returns['Monthly Returns of '+stock].mean()) * beta_vals['Monthly Returns of '+stock].iloc[0])
alpha.head()

,Alpha AAPL,Alpha ABBV,Alpha ABT,Alpha ACN,Alpha AIG,Alpha AMZN,Alpha AXP,Alpha BA,Alpha BAC,Alpha BIIB,...,Alpha SLB,Alpha SO,Alpha SPG,Alpha T,Alpha TGT,Alpha TXN,Alpha UNH,Alpha UNP,Alpha UPS,Alpha USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-06,1.341471,0.512152,0.617273,-0.016195,-3.461665,4.710646,0.504466,-0.058876,-2.906870,-0.965395,...,-5.107431,0.156762,-1.809882,-0.639123,-0.511362,-0.607883,0.110262,-0.462445,-0.068655,-2.872806
2021-07-07,1.665157,0.907275,0.840179,1.178637,-0.078600,0.584902,0.712843,-1.677085,-1.080916,3.159947,...,-1.738918,0.059095,-1.724124,-0.230823,0.802478,-0.825093,0.231641,1.971014,1.450455,-0.198758
2021-07-08,-1.050343,-0.316703,-0.552398,-0.348446,-4.589605,0.960157,-2.159935,2.322183,-2.722566,3.994103,...,-0.655811,-0.248956,-0.831229,-0.730261,0.362983,-0.893866,-0.524990,-4.441120,-0.977315,-1.859789
2021-07-09,1.175136,0.223590,0.358961,0.922535,4.015813,-0.305527,2.526128,1.360303,2.966786,-2.817912,...,1.994907,-0.151988,3.402620,1.037442,-0.053094,0.995997,0.417291,1.476812,0.917154,3.343484
2021-07-12,-0.550751,0.926574,-0.820201,-0.535747,0.751345,-0.003302,0.840838,-0.373318,1.191205,-2.413443,...,-0.070713,0.514829,0.172479,0.184752,1.419278,0.784783,0.844030,-0.269501,-0.230994,0.907687


In [38]:
start = time.perf_counter()
tech_list = []
health_list = []
industrial_list = []
finance_list = []
defensive_list = []
cyclical_list = []
real_estate_list = []
energy_list = []
materials_list = []
communication_list = []
utilities_list = []

for (industry,stock) in zip(industry_list,clean_stocks):
    if industry == "Technology":
        tech_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Healthcare":
        health_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Industrials":
        industrial_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Financial Services":
        finance_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Consumer Defensive":
        defensive_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Consumer Cyclical":
        cyclical_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Real Estate":
        real_estate_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Energy":
        energy_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Basic Materials":
        materials_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Communication Services":
        communication_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
    if industry == "Utilities":
        utilities_list.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),stock))
list_of_industries = [tech_list,health_list,industrial_list,finance_list,defensive_list,cyclical_list,real_estate_list,energy_list,materials_list,communication_list,utilities_list]

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds(s)')

Finished in 0.01 seconds(s)


Our portfolio is divided into different stock market sectors. By the Global Industry Classification Standard (GICS), there are 11 different stock market sectors. As such, the S&P sectors are: Information Technology, Healthcare, Financials, Consumer Discretionary, Communication Services, Industrials, Consumer Staples, Energy, Utilities, Real Estate, and Materials. Thus, the industries categorized within our portfolio reflect the sectors in the S&P500: Technology, Healthcare, Financial Services, Consumer Defensive, Communication Services, Industrials, Consumer Cyclical, Energy, Utilities, Real Estate, and Basic Materials, respectively. 

In [39]:
tech_list.sort()
health_list.sort()
industrial_list.sort()
finance_list.sort()
defensive_list.sort()
cyclical_list.sort()
real_estate_list.sort()
energy_list.sort()
materials_list.sort()
communication_list.sort()
utilities_list.sort()

In [40]:
total_list = []
if len(tech_list)!=0:
    total_list.append(tech_list[0][1])
    
if len(health_list)!=0:
    total_list.append(health_list[0][1])
    
if len(industrial_list)!=0:
    total_list.append(industrial_list[0][1])
    
if len(finance_list)!=0:
    total_list.append(finance_list[0][1])
    
if len(defensive_list)!=0:
    total_list.append(defensive_list[0][1])
    
if len(cyclical_list)!=0:
    total_list.append(cyclical_list[0][1])
    
if len(real_estate_list)!=0:
    total_list.append(real_estate_list[0][1])
    
if len(energy_list)!=0:
    total_list.append(energy_list[0][1])
    
if len(materials_list)!=0:
    total_list.append(materials_list[0][1])
    
if len(communication_list)!=0:
    total_list.append(communication_list[0][1])
    
if len(utilities_list)!=0:
    total_list.append(utilities_list[0][1])
    

In [41]:
total_list 

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO']

In [42]:
alpha_mean = []
alpha = get_alpha(total_list)
alpha.head()

,Alpha CSCO,Alpha BMY,Alpha LMT,Alpha MON,Alpha PG,Alpha SBUX,Alpha SPG,Alpha KMI,Alpha T,Alpha SO
Date,,,,,,,,,,
2021-07-06,-1.134835,-0.324695,-0.731936,0.761834,-0.021393,0.654594,-1.786623,-0.635311,-0.550530,0.148938
2021-07-07,0.439603,0.858574,1.277495,-0.014964,0.669855,1.211892,-1.700865,-0.694482,-0.142230,0.051271
2021-07-08,-0.088893,-0.546842,-0.921076,0.180309,-0.094864,-0.988180,-0.807970,-0.645629,-0.641668,-0.256780
2021-07-09,0.812355,0.828147,0.846902,-0.897100,-0.043761,1.269522,3.425879,2.816546,1.126035,-0.159812
2021-07-12,-1.037911,0.629631,-1.047258,0.367937,0.000017,0.836320,0.195739,-0.738697,0.273345,0.507005


Alpha is used to measure the daily stock return in comparison to the market index that it is compared against. The number output represents a percentage above or below the benchmark index. Note that alpha is calculated using historical closing prices so it does not indicate any future trends. 

In [43]:
mdd = get_mdd(total_list)
mdd

[0.1315745272407484,
 0.18105057874970099,
 0.12263098504021946,
 0.034170870261855966,
 0.08636826787558043,
 0.1305045609406697,
 0.21786154608093122,
 0.17537307844185201,
 0.1154308691397112,
 0.10941946845955973]

Maximum Drawdown (MDD) was used to measure the maximum historical loss and to analyze the downside risk over our 3 month interval. This is used to analyze our stock weighting strategy relative to other weightings, which is one of our primary concerns. For instance, two different portfolio weightings can output the same volatility and performance on average but their maximum drawdowns would be different. This is how we distinguish between a stock with low volatility, consistent average performance, but high maximum drawdown with low volatility, consistent average performance, and low maximum drawdown. As such, low maximum drawdown is always preferred since this deduces that there were minimal losses from the investment. In the case that a maximum drawdown is zero, this would indicate that the stock never lost any money. A stock with 0 maximum drawdown is what we aim to weigh the highest since it entails the least loss. 

Furthermore, in diversifying our portfolio, drawdown risk is reduced; this is because different industry market conditions impact different stocks in different ways. As we explored in Assignment 4, when investing all stocks in the same industry, assets are at a  greater risk of loss. This is why inter-industry investments are a large factor of our safe strategy. by investing in other industries, each stock will react differently to the same market event. For instance, when the market of one industry is growing, the stocks of that industry tend to outperform stocks from other industries. On the contrary, when the market for one industry starts to slow down, stocks from other industries start outperforming this industry’s stocks. as such, by holding inter-industry investments, we are minimizing the fluctuation of our overall portfolio when the market swings one way of another.  

Note that some investors use the equation: (Trough - Peak) / Peak to calculate maximum drawdown (which will entail a negative value). However, since our sole purpose of measuring maximum drawdown is to find the greatest movement from a high point to a low point, we use (Peak - Trough) / Peak ; this is essentially the same as the absolute value of (Trough-Peak)/ Peak. 

However, one limitation of using maximum drawdown is that it does not indicate how long it took for the stock to recover from the loss, or if the investment ever recovers. Additionally, since maximum drawdown only measures the largest drawdown, other smaller periods are ignored— this may cause an inaccurate interpretation of the stock. 

Nevertheless, since our main focus in calculating maximum drawdown is to see the largest deviation between the highest point and the lowest point of the stock, smaller periods of loss/gains are insignificant for our analysis. 


In [44]:
for x in range (len(total_list)):
    clean_stocks.remove(total_list[x])

In [45]:
total_list

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO']

In [46]:
list_of_beta = []
for stock in clean_stocks:
    list_of_beta.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
list_of_beta.sort()
list_of_beta

[(0.7688321597213276, 0.5269870387643538, 'KO'),
 (0.8070678497721258, 0.3509725063939871, 'PEP'),
 (0.9473982893659326, 0.3399536499806572, 'CL'),
 (0.9523078085840201, 0.633750639821062, 'COST'),
 (1.0415130024902401, 0.4585008373597188, 'ABT'),
 (1.046658001376753, 1.0441907562363422, 'ACN'),
 (1.098757173571762, 1.1054609100090915, 'MSFT'),
 (1.1528168917845187, 0.4004039331466627, 'NEE'),
 (1.1976737036390077, 1.2824741684772516, 'GOOG'),
 (1.1998660492312412, 0.754743110750626, 'CVS'),
 (1.2195931497649366, 0.796233841336787, 'MO'),
 (1.2264110981153888, 0.9858310119059345, 'TGT'),
 (1.2565351315580555, 0.6678546757493691, 'PM'),
 (1.2572305193191413, 0.668293470323669, 'ABBV'),
 (1.263780051997507, 1.0655260248317284, 'UNP'),
 (1.2797325890784927, 1.2992799041755803, 'AAPL'),
 (1.3080834411931586, 0.9193301920783156, 'UNH'),
 (1.3491543295133908, 1.0313161135181612, 'TXN'),
 (1.3514651550070829, 0.836516553270562, 'NKE'),
 (1.37991418498633, 0.7130210606742564, 'ORCL'),
 (1.3929

Standard deviation is an effective tool to predict stock trends and measure stock and market volatility. Since we are unwilling to assume risk , a lower standard deviation is preferred and considered when weighing our stocks. A high standard deviation signifies that the magnitude of deviations between closing prices are high, and prices swing widely; the stock is riskier in this sense. As such, standard deviation evaluates the risk and required return on our investment. With our low risk tolerance strategy, we steer toward stocks with lower standard deviation; this also means less variance in monthly returns.

However, there are a few limitation of using standard deviation. Firstly, it is impacted by outliers and extreme values. Standard deviation assumes that the closing prices follow a pattern of normal distribution. As such, standard deviation will give inaccurate results for any stocks that don’t follow a normal pattern of distribution. However, we use standard deviation as a starting point to calculate the risk involved in investing in a particular stock.

In [47]:
threshold = 0
while threshold < len(clean_stocks):
    if len(total_list) == 20:
        break
    beta_temp = get_beta_total(total_list)
    std_temp = get_average_std(total_list)
    beta_stock = list_of_beta[threshold][1]
    std_stock = list_of_beta[threshold][0]
    stock = list_of_beta[threshold][2]
#    print(std_temp,std_stock)
    if beta_stock <= beta_temp and std_stock < std_temp:
        total_list.append(stock)
        clean_stocks.remove(stock)
        list_of_beta.pop(threshold)
        threshold-=1
    threshold+=1
while len(total_list) != 20:
    total_list.append(list_of_beta[0][2])
    list_of_beta.pop(0)
print(total_list)
        

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO', 'KO', 'PEP', 'CL', 'COST', 'ABT', 'ACN', 'MSFT', 'NEE', 'GOOG', 'CVS']


In [48]:
final_returns = get_daily_returns(total_list)
final_returns.head()

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Date,,,,,,,,,,,,,,,,,,,,
2021-07-06,-1.045943,-0.448087,-0.739197,0.776798,0.058867,0.661045,-1.535144,-0.700057,-0.718435,0.178778,-0.553709,0.147743,0.330487,-0.020051,0.660788,0.183690,0.003608,1.144311,0.817286,-1.516448
2021-07-07,0.528496,0.735182,1.270234,0.000000,0.750114,1.218342,-1.449386,-0.759228,-0.310134,0.081111,0.816627,0.442553,0.841774,1.459155,0.883694,1.378523,0.817539,0.266216,0.236190,0.197094
2021-07-08,0.000000,-0.670234,-0.928338,0.195273,-0.014604,-0.981729,-0.556491,-0.710375,-0.809573,-0.226939,-0.349777,0.046739,-0.387135,0.610359,-0.508882,-0.148561,-0.896646,-0.265509,-0.692280,-1.376937
2021-07-09,0.901248,0.704755,0.839640,-0.882137,0.036499,1.275973,3.677358,2.751800,0.958130,-0.129972,0.609640,-0.253569,0.109306,1.282082,0.402476,1.122420,0.187444,-0.186334,0.307715,1.371226
2021-07-12,-0.949018,0.506239,-1.054519,0.382901,0.080277,0.842771,0.447218,-0.803443,0.105440,0.536845,0.036727,0.020062,0.606580,-1.088822,-0.776685,-0.335861,-0.223077,0.133341,0.763655,0.614863


In [49]:
correlation_values = pd.DataFrame(final_returns.corr())
correlation_values.head()

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Monthly Returns of CSCO,1.000000,0.431783,0.270445,0.153152,0.413717,0.332275,0.378010,0.273294,0.412120,0.229745,0.469199,0.375479,0.254952,0.427872,0.345193,0.584371,0.564646,0.190071,0.514603,0.378934
Monthly Returns of BMY,0.431783,1.000000,0.268171,0.032377,0.349729,0.173075,0.170115,0.051525,0.423604,0.273545,0.352593,0.338782,0.301257,0.223678,0.388126,0.420419,0.427358,0.241950,0.190694,0.491485
Monthly Returns of LMT,0.270445,0.268171,1.000000,-0.090981,0.044506,0.266573,0.261668,0.319884,0.323002,0.123064,0.294585,0.233500,0.104034,0.266452,0.047145,0.308594,0.193327,0.003775,0.144177,0.476443
Monthly Returns of MON,0.153152,0.032377,-0.090981,1.000000,0.026129,0.141571,0.013273,-0.041035,0.003940,-0.040426,0.045740,0.071291,-0.020418,0.067134,0.063010,0.119582,0.096793,0.015819,0.159905,0.010602
Monthly Returns of PG,0.413717,0.349729,0.044506,0.026129,1.000000,0.198295,0.074275,-0.002845,0.228961,0.464961,0.552413,0.631309,0.415836,0.323990,0.397196,0.385522,0.322458,0.355242,0.190129,0.253768


In [50]:
final_beta_std = []
for stock in total_list:
    final_beta_std.append((final_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
final_beta_std.sort()
final_beta_std

[(0.4310261262982297, 0.027635681230019388, 'MON'),
 (0.7601216442197638, 0.26914534355453257, 'PG'),
 (0.7688321597213276, 0.5269870387643538, 'KO'),
 (0.8070678497721258, 0.3509725063939871, 'PEP'),
 (0.8252265413854324, 0.2953462792674406, 'SO'),
 (0.8747186606103452, 0.6717403735251262, 'T'),
 (0.9473982893659326, 0.3399536499806572, 'CL'),
 (0.9523078085840201, 0.633750639821062, 'COST'),
 (0.9653513769944979, 0.5331469509993094, 'LMT'),
 (1.0195849623485882, 0.6229328630744411, 'BMY'),
 (1.0335788002409754, 0.8856516761608668, 'CSCO'),
 (1.0415130024902401, 0.4585008373597188, 'ABT'),
 (1.046658001376753, 1.0441907562363422, 'ACN'),
 (1.098757173571762, 1.1054609100090915, 'MSFT'),
 (1.1528168917845187, 0.4004039331466627, 'NEE'),
 (1.1976737036390077, 1.2824741684772516, 'GOOG'),
 (1.1998660492312412, 0.754743110750626, 'CVS'),
 (1.2247158976655264, 0.8139926690008052, 'SBUX'),
 (1.636171983361825, 1.0603002604137866, 'KMI'),
 (1.8787728953059226, 1.5573359928960433, 'SPG')]

In [51]:
print(get_average_std(total_list))
print(get_beta_total(total_list))

1.0431079908984016
0.6817332820531061


In [52]:
list_of_beta = []
for stock in clean_stocks:
    list_of_beta.append((monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
list_of_beta.sort()

In [53]:
threshold = 0
while threshold < len(clean_stocks):
    if len(total_list) == 20:
        break
    beta_temp = get_beta_total(total_list)
    std_temp = get_average_std(total_list)
    beta_stock = list_of_beta[threshold][1]
    std_stock = list_of_beta[threshold][0]
    stock = list_of_beta[threshold][2]
#    print(std_temp,std_stock)
    if std_stock < std_temp:
        if (beta_stock <= beta_temp and beta_temp >= 0) or (beta_stock >= beta_temp and beta_temp < 0):
            total_list.append(stock)
            clean_stocks.remove(stock)
            list_of_beta.pop(threshold)
            threshold-=1
    threshold+=1
counting = 0
while len(total_list) != 20 or len(clean_stocks) ==0 :
    total_list.append(list_of_beta[0][2])
    list_of_beta.pop(0)
print(total_list)

['CSCO', 'BMY', 'LMT', 'MON', 'PG', 'SBUX', 'SPG', 'KMI', 'T', 'SO', 'KO', 'PEP', 'CL', 'COST', 'ABT', 'ACN', 'MSFT', 'NEE', 'GOOG', 'CVS']


In [54]:
final_returns = get_daily_returns(total_list)
final_returns.head()

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Date,,,,,,,,,,,,,,,,,,,,
2021-07-06,-1.045943,-0.448087,-0.739197,0.776798,0.058867,0.661045,-1.535144,-0.700057,-0.718435,0.178778,-0.553709,0.147743,0.330487,-0.020051,0.660788,0.183690,0.003608,1.144311,0.817286,-1.516448
2021-07-07,0.528496,0.735182,1.270234,0.000000,0.750114,1.218342,-1.449386,-0.759228,-0.310134,0.081111,0.816627,0.442553,0.841774,1.459155,0.883694,1.378523,0.817539,0.266216,0.236190,0.197094
2021-07-08,0.000000,-0.670234,-0.928338,0.195273,-0.014604,-0.981729,-0.556491,-0.710375,-0.809573,-0.226939,-0.349777,0.046739,-0.387135,0.610359,-0.508882,-0.148561,-0.896646,-0.265509,-0.692280,-1.376937
2021-07-09,0.901248,0.704755,0.839640,-0.882137,0.036499,1.275973,3.677358,2.751800,0.958130,-0.129972,0.609640,-0.253569,0.109306,1.282082,0.402476,1.122420,0.187444,-0.186334,0.307715,1.371226
2021-07-12,-0.949018,0.506239,-1.054519,0.382901,0.080277,0.842771,0.447218,-0.803443,0.105440,0.536845,0.036727,0.020062,0.606580,-1.088822,-0.776685,-0.335861,-0.223077,0.133341,0.763655,0.614863


In [55]:
correlation_values = pd.DataFrame(final_returns.corr())
correlation_values.head(25)

,Monthly Returns of CSCO,Monthly Returns of BMY,Monthly Returns of LMT,Monthly Returns of MON,Monthly Returns of PG,Monthly Returns of SBUX,Monthly Returns of SPG,Monthly Returns of KMI,Monthly Returns of T,Monthly Returns of SO,Monthly Returns of KO,Monthly Returns of PEP,Monthly Returns of CL,Monthly Returns of COST,Monthly Returns of ABT,Monthly Returns of ACN,Monthly Returns of MSFT,Monthly Returns of NEE,Monthly Returns of GOOG,Monthly Returns of CVS
Monthly Returns of CSCO,1.000000,0.431783,0.270445,0.153152,0.413717,0.332275,0.378010,0.273294,0.412120,0.229745,0.469199,0.375479,0.254952,0.427872,0.345193,0.584371,0.564646,0.190071,0.514603,0.378934
Monthly Returns of BMY,0.431783,1.000000,0.268171,0.032377,0.349729,0.173075,0.170115,0.051525,0.423604,0.273545,0.352593,0.338782,0.301257,0.223678,0.388126,0.420419,0.427358,0.241950,0.190694,0.491485
Monthly Returns of LMT,0.270445,0.268171,1.000000,-0.090981,0.044506,0.266573,0.261668,0.319884,0.323002,0.123064,0.294585,0.233500,0.104034,0.266452,0.047145,0.308594,0.193327,0.003775,0.144177,0.476443
Monthly Returns of MON,0.153152,0.032377,-0.090981,1.000000,0.026129,0.141571,0.013273,-0.041035,0.003940,-0.040426,0.045740,0.071291,-0.020418,0.067134,0.063010,0.119582,0.096793,0.015819,0.159905,0.010602
Monthly Returns of PG,0.413717,0.349729,0.044506,0.026129,1.000000,0.198295,0.074275,-0.002845,0.228961,0.464961,0.552413,0.631309,0.415836,0.323990,0.397196,0.385522,0.322458,0.355242,0.190129,0.253768
Monthly Returns of SBUX,0.332275,0.173075,0.266573,0.141571,0.198295,1.000000,0.308519,0.075928,0.307464,0.254030,0.473341,0.348856,0.310190,0.343247,0.139270,0.439841,0.439257,0.224233,0.462848,0.152265
Monthly Returns of SPG,0.378010,0.170115,0.261668,0.013273,0.074275,0.308519,1.000000,0.507837,0.317789,0.040954,0.263381,0.024235,0.059719,0.100530,-0.071825,0.281304,0.308355,0.102198,0.398576,0.260993
Monthly Returns of KMI,0.273294,0.051525,0.319884,-0.041035,-0.002845,0.075928,0.507837,1.000000,0.348433,-0.135644,0.210492,0.015741,0.132565,-0.051442,-0.168992,0.187555,0.061384,-0.168545,0.297580,0.335658
Monthly Returns of T,0.412120,0.423604,0.323002,0.003940,0.228961,0.307464,0.317789,0.348433,1.000000,0.364275,0.424527,0.263247,0.348614,0.302276,0.166170,0.295334,0.261182,0.289209,0.352181,0.638368
Monthly Returns of SO,0.229745,0.273545,0.123064,-0.040426,0.464961,0.254030,0.040954,-0.135644,0.364275,1.000000,0.520027,0.504707,0.556876,0.312119,0.411579,0.376491,0.291148,0.761784,0.174975,0.228156


In [56]:
final_beta_std = []
list_tuple = []
for stock in total_list:
    final_beta_std.append((final_returns['Monthly Returns of '+stock].std(axis=0,skipna=True),beta_vals["Monthly Returns of "+stock].iloc[0],stock))
final_beta_std.sort(reverse=True)

In [57]:
final_beta_std

[(1.8787728953059226, 1.5573359928960433, 'SPG'),
 (1.636171983361825, 1.0603002604137866, 'KMI'),
 (1.2247158976655264, 0.8139926690008052, 'SBUX'),
 (1.1998660492312412, 0.754743110750626, 'CVS'),
 (1.1976737036390077, 1.2824741684772516, 'GOOG'),
 (1.1528168917845187, 0.4004039331466627, 'NEE'),
 (1.098757173571762, 1.1054609100090915, 'MSFT'),
 (1.046658001376753, 1.0441907562363422, 'ACN'),
 (1.0415130024902401, 0.4585008373597188, 'ABT'),
 (1.0335788002409754, 0.8856516761608668, 'CSCO'),
 (1.0195849623485882, 0.6229328630744411, 'BMY'),
 (0.9653513769944979, 0.5331469509993094, 'LMT'),
 (0.9523078085840201, 0.633750639821062, 'COST'),
 (0.9473982893659326, 0.3399536499806572, 'CL'),
 (0.8747186606103452, 0.6717403735251262, 'T'),
 (0.8252265413854324, 0.2953462792674406, 'SO'),
 (0.8070678497721258, 0.3509725063939871, 'PEP'),
 (0.7688321597213276, 0.5269870387643538, 'KO'),
 (0.7601216442197638, 0.26914534355453257, 'PG'),
 (0.4310261262982297, 0.027635681230019388, 'MON')]

In [58]:
def find_lowest_corr(index,stock):
    lowest_corr = 100
    temp_stock = ""
    for stk in sorted_list:
        if stk != stock:
            curr_corr = correlation_values['Monthly Returns of '+stk].loc['Monthly Returns of '+stock]
            if curr_corr < lowest_corr:
                lowest_corr = curr_corr
                temp_stock = stk
    sorted_list.remove(temp_stock)
    sorted_list.remove(stock)
    list_temp = [stock,temp_stock]
    return list_temp
iterates = len(final_beta_std) - 10
sorted_list = []
list_tuple = []
for x in range (len(final_beta_std)):
    sorted_list.append(final_beta_std[x][2])
for x in range (iterates):
    list_tuple.append(find_lowest_corr(x,sorted_list[0]))
for x in range (len(sorted_list)-1,-1,-1):
    list_tuple.append([sorted_list[x]])
list_tuple

[['SPG', 'ABT'],
 ['KMI', 'NEE'],
 ['SBUX', 'MON'],
 ['CVS', 'GOOG'],
 ['MSFT', 'LMT'],
 ['ACN', 'CL'],
 ['CSCO', 'SO'],
 ['BMY', 'COST'],
 ['T', 'PG'],
 ['PEP', 'KO']]

In [59]:
std_beta_tuple = []
for lists in list_tuple:
    std_beta_tuple.append((get_average_std(lists),get_beta_total(lists),lists))
std_beta_tuple.sort()
std_beta_tuple

[(0.7879500047467267, 0.4389797725791704, ['PEP', 'KO']),
 (0.8174201524150545, 0.47044285853982937, ['T', 'PG']),
 (0.827871011981878, 0.4208141751154123, ['SBUX', 'MON']),
 (0.9294026708132039, 0.5904989777141537, ['CSCO', 'SO']),
 (0.9859463854663042, 0.6283417514477515, ['BMY', 'COST']),
 (0.9970281453713428, 0.6920722031084997, ['ACN', 'CL']),
 (1.03205427528313, 0.8193039305042005, ['MSFT', 'LMT']),
 (1.1987698764351244, 1.0186086396139387, ['CVS', 'GOOG']),
 (1.3944944375731718, 0.7303520967802246, ['KMI', 'NEE']),
 (1.4601429488980813, 1.007918415127881, ['SPG', 'ABT'])]

In [60]:
def create_portfolio(stock_list,weight):
    ticker_frame = pd.DataFrame()
    for x in range (len(stock_list)):
        ticker_frame = pd.concat([ticker_frame, pd.DataFrame((hist_data['Close of '+stock_list[x]]*(1/len(stock_list)*((weight+5)/100)*100000)/hist_data['Close of '+stock_list[x]].iloc[0])).rename(columns={'Close of '+stock_list[x]:'Value of '+stock_list[x]})], axis=1)
    return ticker_frame
def check_fifty(fifty_index,pair_list):
    list_index =[]
    temp_frame = pd.DataFrame()
    temp_beta = pd.DataFrame()
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    for x in range (len(pair_list)):
        if fifty_index == x:
            list_index.append(50)
            temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[x],50)],axis=1)
        else:
            list_index.append(0)
            temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[x],0)],axis=1)
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    temp_frame['Balance'] = temp_frame.sum(axis=1)
    temp_frame['Monthly Returns'] = temp_frame['Balance'].pct_change()* 100
    temp_beta = pd.concat([temp_beta, temp_frame['Monthly Returns']],axis=1)
    MarketVar= temp_beta['^GSPC'].var()
    compare_market = pd.DataFrame(temp_beta.cov() / MarketVar)
    beta_port = compare_market.iloc[0,1]
    alpha_port = (temp_frame['Monthly Returns'] - ((temp_frame['Monthly Returns'].mean())* beta_port)).mean()
    std_port =temp_frame['Monthly Returns'].std(axis=0,skipna=True)
    risk_value = alpha_port+std_port+beta_port
    return (risk_value,list_index)
def check_double(large_index,small_index,big,small,pair_list):
    list_index =[]
    temp_frame = pd.DataFrame()
    temp_beta = pd.DataFrame()
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    temp_frame = create_portfolio(pair_list[large_index],big)
    for y in range (len(pair_list)):
        if y != large_index:
            if small_index == y:
                list_index.append(small)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],small)],axis=1)
            else:
                list_index.append(0)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],0)],axis=1)
        else:
            list_index.append(big)
    temp_frame['Balance'] = temp_frame.sum(axis=1)
    temp_frame['Monthly Returns'] = temp_frame['Balance'].pct_change()* 100
    temp_beta = pd.concat([temp_beta, temp_frame['Monthly Returns']],join='inner',axis=1)
    MarketVar= temp_beta['^GSPC'].var()
    compare_market = pd.DataFrame(temp_beta.cov() / MarketVar)
    beta_port = compare_market.iloc[0,1]
    alpha_port = (temp_frame['Monthly Returns'] - ((temp_frame['Monthly Returns'].mean())* beta_port)).mean()
    std_port =temp_frame['Monthly Returns'].std(axis=0,skipna=True)
    risk_value = alpha_port+std_port+beta_port
    return (risk_value,list_index)     
def check_triple(first,second,third,single,double,pair_list):
    list_index = []
    temp_frame = pd.DataFrame()
    temp_beta = pd.DataFrame()
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    temp_frame = create_portfolio(pair_list[first],single)
    for y in range (len(pair_list)):
        if first!=y:
            if second == y or third == y:
                list_index.append(double)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],double)],axis=1)
            else:
                list_index.append(0)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],0)],axis=1)
        else:
            list_index.append(single)
    temp_frame['Balance'] = temp_frame.sum(axis=1)
    temp_frame['Monthly Returns'] = temp_frame['Balance'].pct_change()* 100
    temp_beta = pd.concat([temp_beta, temp_frame['Monthly Returns']],join='inner',axis=1)
    MarketVar= temp_beta['^GSPC'].var()
    compare_market = pd.DataFrame(temp_beta.cov() / MarketVar)
    beta_port = compare_market.iloc[0,1]
    alpha_port = (temp_frame['Monthly Returns'] - ((temp_frame['Monthly Returns'].mean())* beta_port)).mean()
    std_port =temp_frame['Monthly Returns'].std(axis=0,skipna=True)
    risk_value = alpha_port+std_port+beta_port
    return (risk_value,list_index)     
def check_quadruple(first,second,third,fourth,pair_list):
    list_index =[]
    temp_frame = pd.DataFrame()
    temp_beta = pd.DataFrame()
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    temp_frame = create_portfolio(pair_list[first],20)
    for y in range (len(pair_list)):
        if first != y:
            if second == y or third == y or fourth == y:
                list_index.append(10)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],10)],axis=1)
            else:
                list_index.append(0)
                temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[y],0)],axis=1)
        else:
            list_index.append(20)
    temp_frame['Balance'] = temp_frame.sum(axis=1)
    temp_frame['Monthly Returns'] = temp_frame['Balance'].pct_change()* 100
    temp_beta = pd.concat([temp_beta, temp_frame['Monthly Returns']],join='inner',axis=1)
    MarketVar= temp_beta['^GSPC'].var()
    compare_market = pd.DataFrame(temp_beta.cov() / MarketVar)
    beta_port = compare_market.iloc[0,1]
    alpha_port = (temp_frame['Monthly Returns'] - ((temp_frame['Monthly Returns'].mean())* beta_port)).mean()
    std_port =temp_frame['Monthly Returns'].std(axis=0,skipna=True)
    risk_value = alpha_port+std_port+beta_port
    return (risk_value,list_index)
def check_all(pair_list):
    list_index =[]
    temp_frame = pd.DataFrame()
    temp_beta = pd.DataFrame()
    temp_beta[MarketIndex] = marketing_hist['Close'].pct_change()*100
    for x in range(len(pair_list)):
        list_index.append(10)
        temp_frame = pd.concat([temp_frame,create_portfolio(pair_list[x],10)],axis=1)
    temp_frame['Balance'] = temp_frame.sum(axis=1)
    temp_frame['Monthly Returns'] = temp_frame['Balance'].pct_change()* 100
    temp_beta = pd.concat([temp_beta, temp_frame['Monthly Returns']],join='inner',axis=1)
    MarketVar= temp_beta['^GSPC'].var()
    compare_market = pd.DataFrame(temp_beta.cov() / MarketVar)
    beta_port = compare_market.iloc[0,1]
    alpha_port = (temp_frame['Monthly Returns'] - ((temp_frame['Monthly Returns'].mean())* beta_port)).mean()
    std_port =temp_frame['Monthly Returns'].std(axis=0,skipna=True)
    risk_value = alpha_port+std_port+beta_port
    return (risk_value,list_index)

In [61]:
fifty = []
fourty_ten = []
thirty_twenty = []
ten_twenty_twenty = []
thirty_ten_ten = []
ten_ten_ten_ten_twenty = []
for x in range(len(list_tuple)):
    fifty.append(check_fifty(x,list_tuple))
    for y in range(len(list_tuple)):
        if y != x:
            fourty_ten.append(check_double(x,y,40,10,list_tuple))
            thirty_twenty.append(check_double(x,y,30,20,list_tuple))
            for z in range(len(list_tuple)):
                if z > y and z != x:
                    ten_twenty_twenty.append(check_triple(x,y,z,10,20,list_tuple))
                    thirty_ten_ten.append(check_triple(x,y,z,30,10,list_tuple))
                    for a in range(len(list_tuple)):
                        if a > z and a != x:
                            ten_ten_ten_ten_twenty.append(check_quadruple(x,y,z,a,list_tuple))

In [62]:
all_weights = fifty+fourty_ten+thirty_twenty+ten_twenty_twenty+thirty_ten_ten+ten_ten_ten_ten_twenty
list_of_weights = min(all_weights)[1]
list_of_weights

[0, 0, 30, 0, 0, 0, 0, 0, 20, 0]

In [63]:
def purchase_shares(pair_list,weights):
    ticker_frame = []
    for x in range (len(pair_list)):
        ticker_frame.append((pair_list[x],((1/len(pair_list)*((weights+5)/100)*100000)/hist_data['Close of '+pair_list[x]].iloc[0])))
    return ticker_frame


In [64]:
FinalList = []
for x in range(len(list_tuple)):
    FinalList += purchase_shares(list_tuple[x],list_of_weights[x])
#FinalPortfolio['Balance'] = FinalPortfolio.sum(axis=1)
FinalPortfolio = pd.DataFrame(FinalList)
FinalPortfolio.columns = ['Ticker','Shares']
FinalPortfolio.index+=1

In [65]:
FinalPortfolio.head(25)

,Ticker,Shares
1,SPG,19.503034
2,ABT,21.341977
3,KMI,138.848252
4,NEE,33.960175
5,SBUX,153.463274
6,MON,1812.532417
7,CVS,30.689734
8,GOOG,0.971108
9,MSFT,9.037885
10,LMT,6.600654


In [66]:
FinalPortfolio.to_csv(r'C:Stocks_Group_15.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.